In [1]:
### 只对要提问的句子进行文本扩增
### 先进行回译
### 问句式转换
### 再进行同义词替换

In [2]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk import pos_tag
from nltk.corpus import stopwords
import pandas as pd
import translators as ts
from multiprocessing import Pool
from tqdm import *
from nltk.corpus import wordnet as wn


Using region Guangxi server backend.


In [3]:
import pandas as pd
import numpy as np
post_qa_question = pd.read_excel('post_qa_question_100.xlsx')
questions = []
for i in range(len(post_qa_question)):
    questions.append(post_qa_question['Title'][i])


In [5]:
text = 'What is the difference betwwen C# and Java?'
text

'What is the difference betwwen C# and Java?'

### 先进行回译


In [29]:
import translators as ts
from multiprocessing import Pool
from tqdm import *
text1 = ts.translate_text(query_text = text, translator = 'bing',from_language= 'en',to_language='fi')
text2 = ts.translate_text(query_text = text1, translator = 'bing',from_language= 'fi',to_language='fr')
text3 = ts.translate_text(query_text = text2, translator = 'bing',from_language= 'fr',to_language='en')
print(text3)

What is the difference between C# and Java?


In [28]:
import translators as ts
from multiprocessing import Pool
from tqdm import *
text1 = ts.translate_text(query_text = text, translator = 'bing',from_language= 'en',to_language='fi')
text3 = ts.translate_text(query_text = text1, translator = 'bing',from_language= 'fi',to_language='en')
print(text3)

What is the difference between C# and Java?


首先应该先判断是哪一种问句：
1. 一般疑问句
一般疑问句是最基础的英语疑问句，可以用简单的 yes 或 no 来回答。这类疑问句通常以动词开头——包括助动词（加在主动词之前）和情态动词（比如 can 和 would 等等）。
如果句子里既没有助动词，也没有系动词，改写方式就会跟上面有所不同。我们需要在疑问句的句首加上 do 或者 does，并且要注意相应地调整时态和形式。例如，如果是过去时的话，句首就要用 did。

2. 特殊疑问句
五个以 W 开头的特殊疑问词分别是 who, what, when, where 和 why。不过要注意的是，how 和以 how 开头的一些短语引导的疑问句也属于特殊疑问句。

In [3]:
# pip install gensim
import gensim.downloader as api
model = api.load('glove-twitter-25')  
model.most_similar('awesome', topn=5)



d:\ANACONDA\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


[('amazing', 0.9687871932983398),
 ('best', 0.9600659608840942),
 ('fun', 0.9331521391868591),
 ('fantastic', 0.9313924312591553),
 ('perfect', 0.92434161901474)]

In [6]:
text = word_tokenize(questions[9]) 
text[3]= 'limb'
text_pos = pos_tag(text)
text_pos

[('How', 'WRB'),
 ('do', 'VB'),
 ('you', 'PRP'),
 ('limb', 'VB'),
 ('and', 'CC'),
 ('merge', 'VB'),
 ('with', 'IN'),
 ('TortoiseSVN', 'NNP'),
 ('?', '.')]

In [10]:
# text = word_tokenize(questions[9]) 
# text[3]= 'limb'
text_pos = pos_tag(['happy'])
text_pos

[('happy', 'JJ')]

In [11]:
syn_wordnet('angry')

['tempestuous', 'furious', 'raging', 'wild', 'angry']

In [7]:
def syn_wordnet(word):
    # 通过WordNet获取词语同义词集的方法
    word_synset = set()
    # TODO 获取WordNet中的同义词集
    synsets = wn.synsets(word)  # word所在的词集列表
    for synset in synsets:
        words = synset.lemma_names()
        for word in words:
            word = word.replace('_', ' ')
            word_synset.add(word)
    return list(word_synset)


In [8]:
#对列表的元素进行替换
def list_replace(list,str1,str2):
    list_copy = list.copy()
    for i in range(len(list_copy)):
        if list_copy[i] == str1:
            list_copy[i] = str2
    return list_copy

In [16]:
imp_tag = ['NN','NNS','RB','VB','VBD','VBG','VBN','JJ','JJR','JJS']
ban_word = ['do','can','could','does','did','would','will','have','has','had','must','shall','should']
def synonym_replace_wordnet (text):
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    synonym_sentence = [] #目前的同义句
    #synonym_sentence用来存储当前的句子
    synonym_sentence.append(tokens)
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    for i in range(len(tags)):
        #tags为原始句子当前单词，若为重要标签
        if tags[i][1] in imp_tag and tags[i][0] not in ban_word:
            synonym_tmp = syn_wordnet(tags[i][0])
            synonym = []
            for j in range(len(synonym_tmp)):
                if pos_tag(word_tokenize(synonym_tmp[j]))[0][1] == tags[i][1]:
                    synonym.append(synonym_tmp[j])
            min_time = min(len(synonym),5)
            for j in range(min_time):
                #现在已经有重要词汇替换表了，再用一个for循环来扩充
                for k in range(len(synonym_sentence)):
                    print("synonym_sentence[k]为：",synonym_sentence[k],tokens[i],synonym[j])
                    list = list_replace(synonym_sentence[k],tokens[i],synonym[j])
                    synonym_sentence.append(list)
    return synonym_sentence
        

In [12]:
questions[10]

'How would you access Object properties from within an object method?'

In [22]:
list_synonym = synonym_replace_wordnet(text)

synonym_sentence[k]为： ['What', 'is', 'the', 'difference', 'betwwen', 'C', '#', 'and', 'Java', '?'] difference remainder
synonym_sentence[k]为： ['What', 'is', 'the', 'difference', 'betwwen', 'C', '#', 'and', 'Java', '?'] difference difference
synonym_sentence[k]为： ['What', 'is', 'the', 'remainder', 'betwwen', 'C', '#', 'and', 'Java', '?'] difference difference
synonym_sentence[k]为： ['What', 'is', 'the', 'difference', 'betwwen', 'C', '#', 'and', 'Java', '?'] difference dispute
synonym_sentence[k]为： ['What', 'is', 'the', 'remainder', 'betwwen', 'C', '#', 'and', 'Java', '?'] difference dispute
synonym_sentence[k]为： ['What', 'is', 'the', 'difference', 'betwwen', 'C', '#', 'and', 'Java', '?'] difference dispute
synonym_sentence[k]为： ['What', 'is', 'the', 'remainder', 'betwwen', 'C', '#', 'and', 'Java', '?'] difference dispute
synonym_sentence[k]为： ['What', 'is', 'the', 'difference', 'betwwen', 'C', '#', 'and', 'Java', '?'] difference divergence
synonym_sentence[k]为： ['What', 'is', 'the', 'rem

In [21]:
list_synonym = set([1,5,3,1])

In [15]:
for i in range(len(list_synonym)):
    print(' '.join(list_synonym[i]))

What is the difference betwwen C # and Java ?
What is the remainder betwwen C # and Java ?
What is the difference betwwen C # and Java ?
What is the remainder betwwen C # and Java ?
What is the dispute betwwen C # and Java ?
What is the remainder betwwen C # and Java ?
What is the dispute betwwen C # and Java ?
What is the remainder betwwen C # and Java ?
What is the divergence betwwen C # and Java ?
What is the remainder betwwen C # and Java ?
What is the divergence betwwen C # and Java ?
What is the remainder betwwen C # and Java ?
What is the dispute betwwen C # and Java ?
What is the remainder betwwen C # and Java ?
What is the dispute betwwen C # and Java ?
What is the remainder betwwen C # and Java ?
What is the conflict betwwen C # and Java ?
What is the remainder betwwen C # and Java ?
What is the conflict betwwen C # and Java ?
What is the remainder betwwen C # and Java ?
What is the dispute betwwen C # and Java ?
What is the remainder betwwen C # and Java ?
What is the disput

In [30]:
tt = word_tokenize(questions[9])
tt = pos_tag(tt)

In [31]:
tt

[('How', 'WRB'),
 ('do', 'VB'),
 ('you', 'PRP'),
 ('branch', 'VB'),
 ('and', 'CC'),
 ('merge', 'VB'),
 ('with', 'IN'),
 ('TortoiseSVN', 'NNP'),
 ('?', '.')]

In [62]:
imp_tag = ['NN','NNS','RB','VB','VBD','VBG','VBN','VBP','JJ','JJR','JJS']

def synonym_replace (text):
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    synonym_sentence = [] #目前的同义句
    #synonym_sentence用来存储当前的句子
    synonym_sentence.append(tokens)
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    #对每个标签进行探测，判断是否是重要标签，若为重要，则进行替换，即用小于等于五个词来扩充目前所有句子，可能使元素倍数乘以[1,6]
    for i in range(len(tags)):
        #tags为原始句子当前单词，若为重要标签
        if tags[i][1] in imp_tag:
            synonym = model.most_similar(tags[i][0], topn=5)
            #计算
            for j in range(5):
                if synonym[j][1]>0.92:
                    #现在已经有重要词汇替换表了，再用一个for循环来扩充
                    for k in range(len(synonym_sentence)):
                        # print("synonym_sentence[k]为：",synonym_sentence[k],tokens[i],synonym[j][0])
                        list = list_replace(synonym_sentence[k],tokens[i],synonym[j][0])
                        synonym_sentence.append(list)
    return synonym_sentence
        

In [80]:
text_syn = synonym_replace(text)

In [81]:
text_syn[2]

['What',
 'is',
 'the',
 'important',
 'between',
 'an',
 'int',
 'and',
 'an',
 'Integer',
 'in',
 'Java',
 'and',
 'C',
 '#',
 '?']

### 遮盖词替换

In [66]:
#Masked Language Model 遮盖词替换
from transformers import pipeline
nlp = pipeline('fill-mask')
nlp('This is <mask> cool')


No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'score': 0.5451375246047974,
  'token': 1256,
  'token_str': ' pretty',
  'sequence': 'This is pretty cool'},
 {'score': 0.14360864460468292,
  'token': 269,
  'token_str': ' really',
  'sequence': 'This is really cool'},
 {'score': 0.07481789588928223,
  'token': 2422,
  'token_str': ' super',
  'sequence': 'This is super cool'},
 {'score': 0.05064459890127182,
  'token': 182,
  'token_str': ' very',
  'sequence': 'This is very cool'},
 {'score': 0.022713996469974518,
  'token': 24282,
  'token_str': ' kinda',
  'sequence': 'This is kinda cool'}]

### 进行回译

In [114]:
import translators as ts
from multiprocessing import Pool
from tqdm import *
bq_question = []
text1 = ts.translate_text(query_text = text_tmp, translator = 'bing',from_language= 'en',to_language='cn')
text2 = ts.translate_text(query_text = text1, translator = 'bing',from_language= 'cn',to_language='fr')
text3 = ts.translate_text(query_text = text2, translator = 'bing',from_language= 'fr',to_language='en')
print(text3)

How to use a collection composition as test data


### 测试wordnet

In [8]:
from nltk.corpus import wordnet as wn


In [10]:
synsets = wn.synsets('happy')
print(synsets)


[Synset('happy.a.01'), Synset('felicitous.s.02'), Synset('glad.s.02'), Synset('happy.s.04')]


In [12]:
definition = wn.synset('apple.n.01').definition()
print(definition)


fruit with red or yellow or green skin and sweet to tart crisp whitish flesh


In [13]:
examples = wn.synset('dog.n.01').examples()
examples

['the dog barked all night']

In [18]:
sets = wn.synsets('beau', pos = wn.NOUN)
sets[0].name()

'boyfriend.n.01'

In [20]:
lemma_names = wn.synset('dog.n.01').lemma_names()
lemma_names

['dog', 'domestic_dog', 'Canis_familiaris']

In [7]:
lemma_names = wn.synset(wn.synsets('happy')[0].name()).lemma_names()
lemma_names


['happy']

['well-chosen', 'happy', 'glad', 'felicitous']


In [36]:
from nltk.corpus import wordnet as wn
from nltk.corpus import framenet as fn


def word_synset(word):  # 通过一个关键词，获得该词在Wordnet和framenet的所有同义词
    """
    @parameter : word 关键词
    @return : word_synset 同义词集（或列表）
    """
    word_synset = set()

    # TODO 获取WordNet中的同义词集
    synsets = wn.synsets(word)  # word所在的词集列表
    for synset in synsets:
        words = synset.lemma_names()
        for word in words:
            word = word.replace('_', ' ')
            word_synset.add(word)
#     print(list(word_synset))

    # TODO 获取FrameNet中的词组等
    pattern = r'(?i)%s' % word  # 正则匹配表达式
    frames = fn.frames(pattern) # 获取FrameNet中所有相关框架
    for frame in frames:
        words = [x for x in frame.FE]
        words += words
    for word in words:
        word_synset.add(word)
#     print(list(word_synset))

    word_synset = list(word_synset)
    return word_synset


In [3]:
post_qa_question = pd.read_excel('post_qa_question.xlsx')


In [4]:
post_qa_question

,Unnamed: 0,Answer_Score,Answer_Body,Answer_CommentCount,Id,CreationDate,Score,Body,CommentCount,AcceptedAnswerId,ViewCount,Title,Tags,AnswerCount,FavoriteCount,dect_title_que
0,0,522,an explicit cast to double like this isn't nec...,1,4,2008-07-31T21:42:52.667,794,<p>I want to assign the decimal variable &quot...,8,7.0,70633,How to convert Decimal to Double in C#?,<c#><floating-point><type-conversion><double><...,14,0.0,Question
1,1,160,"the parent div needs to have a defined width, ...",0,6,2008-07-31T22:08:08.620,319,<p>I have an absolutely positioned <code>div</...,0,31.0,23465,Why did the width collapse in the percentage w...,<html><css><internet-explorer-7>,7,0.0,Question
2,2,2385,an easy to understand and simple solution.\n//...,8,9,2008-07-31T23:40:59.743,2199,<p>Given a <code>DateTime</code> representing ...,10,1404.0,784860,How do I calculate someone's age based on a Da...,<c#><.net><datetime>,74,0.0,Question
3,3,222,"the monte carlo method, as mentioned, applies ...",2,19,2008-08-01T05:21:22.257,351,<p>I'm looking for the fastest way to obtain t...,17,531.0,67439,What is the fastest way to get the value of π?,<performance><algorithm><language-agnostic><un...,24,0.0,Question
4,4,45,i've had no trouble using the bsd sockets api ...,0,25,2008-08-01T12:13:50.207,175,<p>I'm having issues getting the C sockets API...,1,1443907.0,15845,How to use the C socket API in C++ on z/OS,<c++><c><sockets><mainframe><zos>,9,0.0,Question
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24532,24532,6,"the h gives it away, in this case the library ...",1,401812,2008-12-30T23:16:24.660,4,<p>What is the proper way to cast from an 'OLE...,0,401879.0,4114,What is the proper way to cast from an 'OLE_HA...,<c++><windows><winapi><mfc><casting>,5,NaN,Question
24533,24533,1,in terms of actually getting the application t...,2,401816,2008-12-30T23:17:53.830,26,<p>How can I make a Click-once deployed app ru...,1,8695121.0,30606,How can I make a Click-once deployed app run a...,<.net><clickonce><startup>,11,0.0,Question
24534,24534,5,"i'm not much of a ruby person either, but as i...",4,401834,2008-12-30T23:27:46.947,3,"<p>Arising out of <a href=""https://stackoverfl...",0,401848.0,2090,How to elegantly compute the anagram signature...,<ruby><algorithm>,3,0.0,Question
24535,24535,4,you can use openssl for that.\n\ntake a look h...,1,401852,2008-12-30T23:36:54.463,4,<p>I'm looking for a good standard based compo...,0,401861.0,4192,Any delphi component for digital signing docum...,<delphi><digital-signature>,3,0.0,Question


In [1]:
a = "sadfasdf"
b = "sadfasdf"
a == b

True